# Background Remover – Android APK Build

This notebook builds the Android APK on Google Colab.

**Run cells in order from top to bottom.**

## 1. Install System Dependencies

Colab is missing some native build tools that python-for-android needs.

In [ ]:
%%bash
# System deps required by p4a / buildozer to compile C extensions (libffi, etc.)
apt-get update -qq
apt-get install -y -qq \
    build-essential \
    git \
    zip \
    unzip \
    openjdk-17-jdk \
    autoconf \
    automake \
    libtool \
    libtool-bin \
    libffi-dev \
    cmake \
    libltdl-dev \
    pkg-config \
    zlib1g-dev \
    libncurses5-dev \
    libncursesw5-dev \
    libtinfo5 \
    libgdbm-dev \
    libssl-dev \
    libreadline-dev \
    libsqlite3-dev \
    tk-dev \
    lzma \
    liblzma-dev \
    libgdbm-compat-dev \
    ccache \
    2>&1 | tail -3

# ─── CRITICAL FIX: make libtool m4 macros visible to aclocal ───
# Without this, autoreconf cannot resolve AC_PROG_LIBTOOL in libffi's configure.ac
ACLOCAL_SYS="/usr/share/aclocal"
ACLOCAL_LOCAL="/usr/local/share/aclocal"
mkdir -p "$ACLOCAL_LOCAL"
for f in "$ACLOCAL_SYS"/libtool.m4 "$ACLOCAL_SYS"/lt*.m4; do
    [ -f "$f" ] && cp -f "$f" "$ACLOCAL_LOCAL/"
done
echo "libtool m4 macros copied to $ACLOCAL_LOCAL:"
ls "$ACLOCAL_LOCAL"/lt*.m4 "$ACLOCAL_LOCAL"/libtool.m4 2>/dev/null

echo '\n✅ System dependencies installed.'

## 2. Install Buildozer & Cython

In [ ]:
!pip install -q buildozer cython==3.0.11
!buildozer version 2>/dev/null || echo "buildozer installed (no version cmd)"
print('✅ Buildozer + Cython ready')

## 3. Clone the Repository

In [ ]:
import os

REPO = 'https://github.com/nawaz0x1/Background-Remover-App.git'
PROJECT_DIR = '/content/Background-Remover-App'

if os.path.isdir(PROJECT_DIR):
    os.chdir(PROJECT_DIR)
    !git pull
else:
    !git clone {REPO} {PROJECT_DIR}
    os.chdir(PROJECT_DIR)

print(f'\n✅ Working directory: {os.getcwd()}')
!ls -la

## 4. Download the U2Net ONNX Model

The model (~176 MB) is bundled inside the APK.

In [ ]:
!python download_model.py

## 5. Clean Previous Build (run this if a previous build failed!)

Removes the stale build tree so libffi gets rebuilt from scratch with the fixed libtool macros.

In [ ]:
# Delete only the broken libffi build (fast) instead of nuking everything
!rm -rf .buildozer/android/platform/build-arm64-v8a/build/other_builds/libffi
# Uncomment below for a FULL clean rebuild (slow – re-downloads SDK/NDK):
# !rm -rf .buildozer

## 6. Build the APK

This takes **15-30 minutes** on a fresh build (SDK/NDK download + compilation).
Subsequent builds are much faster.

In [ ]:
import os
# Ensure libtool m4 macros are findable by aclocal in all subprocesses
os.environ['ACLOCAL_PATH'] = '/usr/local/share/aclocal:/usr/share/aclocal'
!buildozer -v android debug 2>&1 | tee build.log | tail -50

## 7. Download the APK

In [ ]:
import glob
from google.colab import files

apks = glob.glob('bin/*.apk')
if apks:
    apk = apks[-1]  # latest
    print(f'✅ APK ready: {apk}  ({os.path.getsize(apk) / 1e6:.1f} MB)')
    files.download(apk)
else:
    print('❌ No APK found. Check build.log for errors:')
    !tail -30 build.log

## Troubleshooting

If the build fails, check the full log:

In [ ]:
# Show last 100 lines of build log
!tail -100 build.log